Connor Krill Updated July 18, 2019
# Automating the running and post processing of hyades
This script runs the hyades simulation, post-processing on all variables of interest, and puts all of those files into a single folder for each hyades simulation. A log file will be created and time stamp every hyades run this script preforms.

In [ ]:
import os
import shutil
import time
import logging
import matplotlib.pyplot as plt
from tools_hyades_runner import run_hyades, run_hyades_post
from tools_excel_writer import write_excel

# Setup information for the hyades run
* inf_path - the location of all your .inf files. Everything in this folder with the extension ".inf" will be run.
* final_destination - where all of the data will be saved and all these files will end up.
* copy_data_to_excel - __Recommended set to 'True' if you want to plot the data outside of python__ if True an excel sheet with all of the data is saved for each simulation. Each column is header is a point in time and each row is a lagranian position X. The corresponding table value is the variable value at that point in time and distance. 


In [ ]:
inf_path = '../data/inf'
final_destination = '../data'
copy_data_to_excel = True

In [ ]:
variables = ['Pres', 'Rho', 'Te', 'U']
# setup a logging file
filename   = 'hyades.log'
log_format = '%(asctime)s %(levelname)s:%(message)s'
datefmt    = '%Y-%m-%d %H:%M:%S'
logging.basicConfig(filename=filename, 
                    format=log_format, datefmt=datefmt, level=logging.DEBUG)
error_str = f'{filename} not in current directory, {os.getcwd()}'
assert filename in os.listdir(os.getcwd()), error_str

old_dir = os.getcwd() # save the old directory in case I need it later

inf_files = sorted([os.path.splitext(f)[0] for f in os.listdir(inf_path)
                   if (f.endswith('.inf')) and ('setup' not in f)])

if inf_files:
    print(f'Found {len(inf_files)} .inf files: {", ".join(inf_files)} \n')
else:
    raise Exception(f'Found no .inf files in {inf_path!r}')
                    
for run_name in inf_files:

    t2complete = run_hyades(inf_path, run_name, final_destination) # creates a new folder named "run_name"
    print(t2complete)
    logging.info(f'Complete HYADES run for {run_name} in {round(t2complete,1)} seconds')
    os.chdir(run_name)
    run_hyades_post(run_name, variables) # post processing on all variables
    logging.info(f'Completed post processing for {run_name} variables: {", ".join(variables)}')
    
    if copy_data_to_excel:
        write_excel(run_name, run_name, variables)
    
    os.chdir(old_dir)
    
    # move the folder and post processing to final_destination
    shutil.move(os.path.join(inf_path, run_name), final_destination) 
    print()

print('\nFinished')